In [1]:
from keras.datasets import boston_housing
(train_data, train_target), (test_data, test_target) = boston_housing.load_data()

In [2]:
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std

test_data -= mean
test_data /= std

In [3]:
from keras import models
from keras import layers

def build_model() :
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(train_data.shape[1], )))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

In [4]:
k = 4
n = len(train_data) // k
epochs = 100
all_scores = []

In [5]:
import numpy as np

for i in range(k) :
    print('processing fold:', i)
    val_data = train_data[n * i : n * (i + 1)]
    val_target = train_target[n * i : n * (i + 1)]
    
    partial_train_data = np.concatenate([train_data[:n * i],train_data[n * (i + 1):]], axis=0)
    partial_train_target = np.concatenate([train_target[:n * i],train_target[n * (i + 1):]], axis=0)
    
    model = build_model()
    model.fit(partial_train_data, partial_train_target, epochs=epochs, batch_size=1, verbose=0)
    val_mse, val_mae = model.evaluate(val_data, val_target, verbose=0)
    all_scores.append(val_mae)

processing fold: 0
processing fold: 1
processing fold: 2
processing fold: 3


In [6]:
print(all_scores)
print(np.mean(all_scores))

[2.135653018951416, 3.0914225578308105, 2.5584053993225098, 2.213132858276367]
2.499653458595276


In [7]:
test_mse, test_mae = model.evaluate(test_data, test_target)

4/4 [==============================] - 0s 644us/step - loss: 16.1261 - mae: 2.4609
